In [1]:
import spacy
import pandas as pd
import plotly.express as pe

In [2]:
nlp=spacy.load("output/model-last/")

In [3]:
df=pd.read_excel("registo_consultas_teste.xlsx")

In [4]:
df.head()

,ID,Nota
0,1,Doente bem. Está actualmente a fazer Enzalutam...
1,2,"Hoje vem apenas o marido. Doente mal, em progr..."
2,3,Tudo ok. Vai manter trifluridina+tipiracilo 1x...
3,4,Vem a própria. Refere cefaleias e insónias oca...
4,5,"A fazer Osimertinib 80mg/dia. Animada, sem pro..."


In [14]:
df.ID.nunique()


5

In [5]:
ent = []
labels = []
def processar(x):
    doc = nlp(x)
    for entity in doc.ents:
        ent.append(entity.text)
        labels.append(entity.label_)

In [6]:
df["Nota"].apply(processar)

0    None
1    None
2    None
3    None
4    None
Name: Nota, dtype: object

In [7]:
ent

['bem',
 'Enzalutamida',
 'rash',
 'TAC',
 'mal',
 'em progressão',
 'capecitabina',
 '1500mg',
 'bid',
 'diarreia',
 'vómitos',
 'ok',
 '1xdia',
 'insónias',
 'paracetamol',
 'melhor',
 'Osimertinib',
 '80mg',
 '/dia',
 'Animada',
 'sem progressão',
 'náuseas']

In [8]:
base = pd.DataFrame({"Entidade":ent,"Classificação":labels})
base

,Entidade,Classificação
0,bem,Estado
1,Enzalutamida,Terapêutica
2,rash,RAM
3,TAC,MCDT
4,mal,Estado
5,em progressão,Estado
6,capecitabina,Terapêutica
7,1500mg,Dose
8,bid,Posologia
9,diarreia,RAM


In [9]:
RAMS=base.loc[base.Classificação=="RAM"]
pe.pie(RAMS, names=RAMS.Entidade, title = "Distribuição de RAMS", width = 400, height = 400)

In [10]:
doc=nlp(texto)

NameError: name 'texto' is not defined

In [ ]:
from spacy import displacy
colors = {"Estado" : "#5F9EA0", "Terapêutica" : "#00FF00","RAM" : "#FFA07A","MCDT" : "#FF00FF", "Dose" : "#696969", "Posologia" : "#E6E6FA"}
options = {"colors" : colors}
displacy.render(doc,style="ent",jupyter="True", options = options)

In [20]:
%%writefile app.py
import streamlit as st
import pandas as pd
import plotly.express as pe
import spacy
import openpyxl
from spacy import displacy

def text_function(texto):
    nlp = spacy.load("output/model-last/")
    doc = nlp(texto)
    colors = {"Estado" : "#5F9EA0", "Terapêutica" : "#00FF00","RAM" : "#FFA07A","MCDT" : "#FF00FF", "Dose" : "#696969", "Posologia" : "#E6E6FA"}
    options = {"colors" : colors}
    html = displacy.render(doc,style = "ent", jupyter = False,options = options)
    st.markdown(html, unsafe_allow_html = True)

ent = []
labels = []
def processar(x):
    nlp = spacy.load("output/model-last/")
    doc = nlp(x)
    for entity in doc.ents:
        ent.append(entity.text)
        labels.append(entity.label_)

def main():
    st.set_page_config(layout = 'wide', initial_sidebar_state = 'expanded')
    st.title('NLP em consulta farmacêutica')
    
    st.header("Utilizando um ficheiro")
    ficheiro = st.file_uploader("Faça Upload do ficheiro contendo as notas clínicas", type=["xls","xlsx"])
    if ficheiro is not None:
        file_data =  ficheiro.read()
        st.write("Ficheiro escolhido:", ficheiro.name)
        df=pd.read_excel(ficheiro)
        
    st.header("Utilizando texto livre")
    texto = st.text_input("Insira aqui a nota")

    if st.button("Analisar"):
        if ficheiro is not None:
            st.header("Dashboard de análise")
            st.write("Foram analisados registos de **{}** doentes".format(df.ID.nunique()))
            df["Nota"].apply(processar)
            base = pd.DataFrame({"Entidade":ent,"Classificação":labels})
            RAMS=base.loc[base.Classificação=="RAM"]
            fig1=pe.pie(RAMS, names=RAMS.Entidade, title = "Distribuição de RAMS", width = 400, height = 400)
            Tx=base.loc[base.Classificação=="Terapêutica"]
            fig2=pe.pie(Tx, names=Tx.Entidade, title = "Distribuição de Terapêutica", width = 400, height = 400)
            st.plotly_chart(fig1), st.plotly_chart(fig2)
            
        else:
            return text_function(texto)
if __name__ == "__main__":
    main()

Overwriting app.py
